# Day 11: Google Managed MCP

**Connect Agents to Google Services with Managed MCP.**

## 课程来源
- [Advent of Agents 2025 - Day 11](https://adventofagents.com/)
- [Announcing Official MCP Support for Google Services](https://cloud.google.com/blog/products/ai-machine-learning/announcing-official-mcp-support-for-google-services)
- [ADK MCP Documentation](https://google.github.io/adk-docs/mcp/)
- [Kaggle Whitepaper: Agent Tools and Interoperability with MCP](https://www.kaggle.com/whitepaper-agent-tools-and-interoperability-with-mcp)

## Learning Goals

1. 理解 MCP (Model Context Protocol) 是什么
2. 学习 Google Managed MCP 服务器的优势
3. 掌握 ADK 中 McpToolset 的使用方法
4. 实践连接 Google 服务（Maps, BigQuery 等）

---
## 1. 什么是 MCP (Model Context Protocol)？

### "USB-C for AI"

MCP 是由 Anthropic 开发的开源标准协议，用于连接 AI 模型与数据/工具。

```
传统方式：每个 AI 应用需要为每个工具写专门的集成代码

┌─────────┐     ┌─────────┐     ┌─────────┐
│ Agent A │────▶│ Tool 1  │     │ Tool 2  │
└─────────┘     └─────────┘     └─────────┘
     │              ▲               ▲
     │              │               │
     └──────────────┴───────────────┘
         自定义代码 × N × M

MCP 方式：统一协议，即插即用

┌─────────┐                      ┌─────────┐
│ Agent A │──┐                ┌──│ Tool 1  │
└─────────┘  │                │  └─────────┘
             │  ┌──────────┐  │
┌─────────┐  ├──│   MCP    │──┤  ┌─────────┐
│ Agent B │──┤  │ Protocol │  ├──│ Tool 2  │
└─────────┘  │  └──────────┘  │  └─────────┘
             │                │
┌─────────┐  │                │  ┌─────────┐
│ Agent C │──┘                └──│ Tool 3  │
└─────────┘                      └─────────┘
```

> MCP 让任何 AI Agent 可以连接任何 MCP 兼容的工具，就像 USB-C 让设备可以连接任何配件一样。

### MCP 的核心概念

| 概念 | 说明 |
|------|------|
| **MCP Server** | 暴露工具/资源的服务端 |
| **MCP Client** | 连接服务端的客户端（如 AI Agent） |
| **Tools** | 服务端提供的可调用功能 |
| **Resources** | 服务端提供的数据访问 |
| **Prompts** | 服务端提供的 prompt 模板 |

### 连接方式

| 类型 | 说明 | 使用场景 |
|------|------|----------|
| **Stdio** | 通过标准输入/输出通信 | 本地 MCP 服务器 |
| **SSE** | Server-Sent Events | 远程服务器（单向） |
| **StreamableHTTP** | HTTP 流式传输 | 远程托管服务器 |

---
## 2. Google Managed MCP 服务器

### 传统 MCP vs Google Managed MCP

| 传统方式 | Google Managed MCP |
|----------|--------------------|
| 需要手动部署 MCP 服务器 | Google 托管，即用即得 |
| 自己管理安全和认证 | Cloud IAM 权限控制 |
| 自己处理扩展和运维 | 自动扩展，免运维 |
| 可能存在安全漏洞 | Model Armor 防护 |

### 首批支持的 Google 服务

| 服务 | 功能 |
|------|------|
| **Google Maps** | 地图、导航、地点搜索 |
| **BigQuery** | 数据查询、分析 |
| **Compute Engine** | 虚拟机管理 |
| **Kubernetes Engine** | 容器编排 |
| **Cloud Run** | 无服务器容器 |
| **Cloud Storage** | 对象存储 |
| **AlloyDB / Cloud SQL / Spanner** | 数据库服务 |
| **Looker** | 商业智能 |
| **Pub/Sub** | 消息队列 |
| **Dataplex** | 数据治理 |

> "We are making Google agent-ready by design." - Steren Giannini, Google Cloud

### 安全特性

```
┌─────────────────────────────────────────────────────────────┐
│                    Google Managed MCP                        │
├─────────────────────────────────────────────────────────────┤
│                                                              │
│  ┌──────────────────┐    ┌──────────────────┐              │
│  │   Cloud IAM      │    │   Model Armor    │              │
│  │                  │    │                  │              │
│  │ • 身份验证       │    │ • Prompt 注入防护│              │
│  │ • 权限控制       │    │ • 数据泄露防护   │              │
│  │ • 审计日志       │    │ • 恶意请求过滤   │              │
│  └──────────────────┘    └──────────────────┘              │
│                                                              │
│  ┌──────────────────────────────────────────┐              │
│  │              数据治理                     │              │
│  │                                          │              │
│  │ • 数据不离开 Google Cloud                │              │
│  │ • 遵循企业数据策略                       │              │
│  │ • 支持 VPC Service Controls              │              │
│  └──────────────────────────────────────────┘              │
│                                                              │
└─────────────────────────────────────────────────────────────┘
```

---
## 3. Setup

In [1]:
# 安装依赖
# !uv pip install google-adk google-genai mcp python-dotenv

import sys
from pathlib import Path

# Add project root to path
sys.path.insert(0, str(Path.cwd().parent))

from shared import get_api_key

# 加载 API key
GOOGLE_API_KEY = get_api_key()
print("✅ API Key loaded")

✅ API Key loaded


---
## 4. ADK 中的 McpToolset

ADK 提供 `McpToolset` 类来集成 MCP 服务器的工具。

### McpToolset 的工作原理

```
┌─────────────────────────────────────────────────────────────┐
│                     ADK Agent                                │
│  ┌──────────────────────────────────────────────────────┐  │
│  │                   McpToolset                          │  │
│  │                                                       │  │
│  │  1. 连接 MCP 服务器                                   │  │
│  │  2. 调用 list_tools 获取可用工具                     │  │
│  │  3. 转换为 ADK BaseTool 格式                         │  │
│  │  4. 代理工具调用到 MCP 服务器                        │  │
│  │  5. 返回结果给 Agent                                 │  │
│  │                                                       │  │
│  └───────────────────────┬──────────────────────────────┘  │
└──────────────────────────┼──────────────────────────────────┘
                           │
                           ▼
               ┌───────────────────────┐
               │     MCP Server        │
               │  (Maps, BigQuery...)  │
               └───────────────────────┘
```

### 基本用法

In [2]:
# McpToolset 的基本结构（概念示例）

from google.adk.agents import LlmAgent
from google.adk.tools.mcp_tool import McpToolset

# 查看 McpToolset 的文档
print("McpToolset 主要参数：")
print("""
McpToolset(
    connection_params=...,   # 连接参数（Stdio/SSE/HTTP）
    tool_filter=[...],       # 可选：只使用部分工具
)
""")

McpToolset 主要参数：

McpToolset(
    connection_params=...,   # 连接参数（Stdio/SSE/HTTP）
    tool_filter=[...],       # 可选：只使用部分工具
)



---
## 5. 连接方式详解

### 方式 1: Stdio 连接（本地 MCP 服务器）

In [3]:
# Stdio 连接示例 - 连接本地 MCP 服务器

from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

# 示例：连接 Google Maps MCP 服务器（需要 npx 和 API key）
stdio_example = """
import os
from google.adk.agents import LlmAgent
from google.adk.tools.mcp_tool import McpToolset
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

google_maps_api_key = os.getenv("GOOGLE_MAPS_API_KEY")

maps_agent = LlmAgent(
    model='gemini-2.0-flash',
    name='maps_assistant',
    instruction='Help users with maps, directions, and places.',
    tools=[
        McpToolset(
            connection_params=StdioConnectionParams(
                server_params=StdioServerParameters(
                    command='npx',
                    args=["-y", "@modelcontextprotocol/server-google-maps"],
                    env={"GOOGLE_MAPS_API_KEY": google_maps_api_key}
                ),
            ),
            tool_filter=['get_directions', 'find_place']  # 只使用这些工具
        )
    ],
)
"""

print("Stdio 连接示例（本地 MCP 服务器）：")
print(stdio_example)

Stdio 连接示例（本地 MCP 服务器）：

import os
from google.adk.agents import LlmAgent
from google.adk.tools.mcp_tool import McpToolset
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

google_maps_api_key = os.getenv("GOOGLE_MAPS_API_KEY")

maps_agent = LlmAgent(
    model='gemini-2.0-flash',
    name='maps_assistant',
    instruction='Help users with maps, directions, and places.',
    tools=[
        McpToolset(
            connection_params=StdioConnectionParams(
                server_params=StdioServerParameters(
                    command='npx',
                    args=["-y", "@modelcontextprotocol/server-google-maps"],
                    env={"GOOGLE_MAPS_API_KEY": google_maps_api_key}
                ),
            ),
            tool_filter=['get_directions', 'find_place']  # 只使用这些工具
        )
    ],
)



### 方式 2: StreamableHTTP 连接（远程托管服务器）

In [4]:
# StreamableHTTP 连接示例 - 连接远程 MCP 服务器

http_example = """
import os
from google.adk.agents import LlmAgent
from google.adk.tools.mcp_tool import McpToolset, StreamableHTTPConnectionParams

currency_agent = LlmAgent(
    model="gemini-2.5-flash",
    name="currency_agent",
    description="An agent that helps with currency conversions",
    instruction="You are a specialized assistant for currency conversions.",
    tools=[
        McpToolset(
            connection_params=StreamableHTTPConnectionParams(
                url=os.getenv("MCP_SERVER_URL", "http://localhost:8080/mcp")
            )
        )
    ],
)
"""

print("StreamableHTTP 连接示例（远程 MCP 服务器）：")
print(http_example)

StreamableHTTP 连接示例（远程 MCP 服务器）：

import os
from google.adk.agents import LlmAgent
from google.adk.tools.mcp_tool import McpToolset, StreamableHTTPConnectionParams

currency_agent = LlmAgent(
    model="gemini-2.5-flash",
    name="currency_agent",
    description="An agent that helps with currency conversions",
    instruction="You are a specialized assistant for currency conversions.",
    tools=[
        McpToolset(
            connection_params=StreamableHTTPConnectionParams(
                url=os.getenv("MCP_SERVER_URL", "http://localhost:8080/mcp")
            )
        )
    ],
)



### 方式 3: SSE 连接（Server-Sent Events）

In [5]:
# SSE 连接示例

sse_example = """
from google.adk.tools.mcp_tool import McpToolset, SseConnectionParams

# SSE 连接参数
sse_tools = McpToolset(
    connection_params=SseConnectionParams(
        url="http://localhost:8001/sse"
    )
)
"""

print("SSE 连接示例：")
print(sse_example)

SSE 连接示例：

from google.adk.tools.mcp_tool import McpToolset, SseConnectionParams

# SSE 连接参数
sse_tools = McpToolset(
    connection_params=SseConnectionParams(
        url="http://localhost:8001/sse"
    )
)



---
## 6. 实战示例：模拟 MCP 集成

由于实际的 MCP 服务器需要额外配置，我们先用模拟的方式理解概念。

In [6]:
# 模拟 MCP 工具的工作方式

from typing import Any, Dict, List
import json

class MockMcpServer:
    """模拟 MCP 服务器"""
    
    def __init__(self, name: str):
        self.name = name
        self._tools = {}
    
    def register_tool(self, name: str, description: str, handler):
        """注册工具"""
        self._tools[name] = {
            "name": name,
            "description": description,
            "handler": handler
        }
    
    def list_tools(self) -> List[Dict]:
        """MCP list_tools 方法"""
        return [
            {"name": t["name"], "description": t["description"]}
            for t in self._tools.values()
        ]
    
    def call_tool(self, name: str, arguments: Dict) -> Any:
        """MCP call_tool 方法"""
        if name not in self._tools:
            raise ValueError(f"Tool {name} not found")
        return self._tools[name]["handler"](**arguments)


# 创建模拟的 Google Maps MCP 服务器
maps_server = MockMcpServer("google-maps")

# 注册工具
def get_directions(origin: str, destination: str, mode: str = "driving") -> Dict:
    """Get directions between two locations."""
    # 模拟返回
    return {
        "origin": origin,
        "destination": destination,
        "mode": mode,
        "distance": "15.2 km",
        "duration": "23 mins",
        "steps": [
            "Head north on Main St",
            "Turn right onto Highway 101",
            "Take exit 12B toward Downtown",
            "Arrive at destination"
        ]
    }

def find_place(query: str, location: str = None) -> Dict:
    """Find places matching the query."""
    # 模拟返回
    return {
        "query": query,
        "results": [
            {"name": f"{query} A", "rating": 4.5, "address": "123 Main St"},
            {"name": f"{query} B", "rating": 4.2, "address": "456 Oak Ave"},
        ]
    }

maps_server.register_tool("get_directions", "Get directions between locations", get_directions)
maps_server.register_tool("find_place", "Find places matching a query", find_place)

print("✅ Mock MCP Server created: google-maps")
print("\nAvailable tools:")
for tool in maps_server.list_tools():
    print(f"  - {tool['name']}: {tool['description']}")

✅ Mock MCP Server created: google-maps

Available tools:
  - get_directions: Get directions between locations
  - find_place: Find places matching a query


In [7]:
# 模拟 McpToolset 的工作方式

class MockMcpToolset:
    """模拟 ADK 的 McpToolset"""
    
    def __init__(self, server: MockMcpServer, tool_filter: List[str] = None):
        self.server = server
        self.tool_filter = tool_filter
        self._cached_tools = None
    
    def get_tools(self) -> List[Dict]:
        """获取可用工具（转换为 ADK 格式）"""
        if self._cached_tools is None:
            all_tools = self.server.list_tools()
            if self.tool_filter:
                all_tools = [t for t in all_tools if t["name"] in self.tool_filter]
            self._cached_tools = all_tools
        return self._cached_tools
    
    def call(self, tool_name: str, **kwargs) -> Any:
        """调用工具（代理到 MCP 服务器）"""
        print(f"📡 McpToolset: Calling {tool_name} on {self.server.name}")
        result = self.server.call_tool(tool_name, kwargs)
        print(f"✅ McpToolset: Got response from {self.server.name}")
        return result


# 创建 toolset
maps_toolset = MockMcpToolset(maps_server)

print("Tools available through McpToolset:")
for tool in maps_toolset.get_tools():
    print(f"  - {tool['name']}")

Tools available through McpToolset:
  - get_directions
  - find_place


In [8]:
# 模拟 Agent 使用 MCP 工具

print("\n" + "=" * 60)
print("Simulating Agent using MCP tools")
print("=" * 60)

# 场景 1: 获取路线
print("\n📍 User: How do I get from SF to San Jose?")
print("\n🤖 Agent: Let me get directions for you...")

directions = maps_toolset.call(
    "get_directions",
    origin="San Francisco, CA",
    destination="San Jose, CA"
)

print(f"\n🗺️ Directions from {directions['origin']} to {directions['destination']}:")
print(f"   Distance: {directions['distance']}")
print(f"   Duration: {directions['duration']}")
print("   Steps:")
for i, step in enumerate(directions['steps'], 1):
    print(f"     {i}. {step}")


Simulating Agent using MCP tools

📍 User: How do I get from SF to San Jose?

🤖 Agent: Let me get directions for you...
📡 McpToolset: Calling get_directions on google-maps
✅ McpToolset: Got response from google-maps

🗺️ Directions from San Francisco, CA to San Jose, CA:
   Distance: 15.2 km
   Duration: 23 mins
   Steps:
     1. Head north on Main St
     2. Turn right onto Highway 101
     3. Take exit 12B toward Downtown
     4. Arrive at destination


In [9]:
# 场景 2: 搜索地点
print("\n📍 User: Find me coffee shops nearby")
print("\n🤖 Agent: Searching for coffee shops...")

places = maps_toolset.call(
    "find_place",
    query="Coffee Shop",
    location="San Francisco"
)

print(f"\n☕ Found {len(places['results'])} coffee shops:")
for place in places['results']:
    print(f"   - {place['name']} (⭐ {place['rating']}) - {place['address']}")


📍 User: Find me coffee shops nearby

🤖 Agent: Searching for coffee shops...
📡 McpToolset: Calling find_place on google-maps
✅ McpToolset: Got response from google-maps

☕ Found 2 coffee shops:
   - Coffee Shop A (⭐ 4.5) - 123 Main St
   - Coffee Shop B (⭐ 4.2) - 456 Oak Ave


---
## 7. BigQuery MCP 示例

BigQuery MCP 服务器让 Agent 可以直接查询企业数据。

In [10]:
# 模拟 BigQuery MCP 服务器

bigquery_server = MockMcpServer("bigquery")

# 模拟数据
MOCK_DATABASE = {
    "sales": [
        {"date": "2024-01-01", "product": "Widget A", "amount": 1500},
        {"date": "2024-01-02", "product": "Widget B", "amount": 2300},
        {"date": "2024-01-03", "product": "Widget A", "amount": 1800},
    ],
    "customers": [
        {"id": 1, "name": "Alice", "tier": "premium"},
        {"id": 2, "name": "Bob", "tier": "standard"},
    ]
}

def list_datasets() -> Dict:
    """List available datasets."""
    return {"datasets": list(MOCK_DATABASE.keys())}

def get_schema(dataset: str) -> Dict:
    """Get schema of a dataset."""
    if dataset not in MOCK_DATABASE:
        return {"error": f"Dataset {dataset} not found"}
    sample = MOCK_DATABASE[dataset][0]
    return {
        "dataset": dataset,
        "columns": list(sample.keys()),
        "row_count": len(MOCK_DATABASE[dataset])
    }

def run_query(query: str) -> Dict:
    """Run a SQL query (mock implementation)."""
    # 简化的查询解析
    if "sales" in query.lower():
        total = sum(row["amount"] for row in MOCK_DATABASE["sales"])
        return {
            "query": query,
            "result": [{"total_sales": total}],
            "rows_returned": 1
        }
    return {"error": "Query not supported in mock"}

bigquery_server.register_tool("list_datasets", "List available datasets", list_datasets)
bigquery_server.register_tool("get_schema", "Get dataset schema", get_schema)
bigquery_server.register_tool("run_query", "Run a SQL query", run_query)

bq_toolset = MockMcpToolset(bigquery_server)

print("✅ Mock BigQuery MCP Server created")
print("\nAvailable tools:")
for tool in bq_toolset.get_tools():
    print(f"  - {tool['name']}: {tool['description']}")

✅ Mock BigQuery MCP Server created

Available tools:
  - list_datasets: List available datasets
  - get_schema: Get dataset schema
  - run_query: Run a SQL query


In [11]:
# 模拟 Agent 使用 BigQuery MCP

print("\n" + "=" * 60)
print("Simulating Data Analysis Agent")
print("=" * 60)

print("\n📊 User: What data do we have available?")
print("\n🤖 Agent: Let me check the available datasets...")

datasets = bq_toolset.call("list_datasets")
print(f"\n📁 Available datasets: {datasets['datasets']}")

print("\n📊 User: Tell me about the sales data")
print("\n🤖 Agent: Getting schema for sales dataset...")

schema = bq_toolset.call("get_schema", dataset="sales")
print(f"\n📋 Sales dataset schema:")
print(f"   Columns: {schema['columns']}")
print(f"   Rows: {schema['row_count']}")

print("\n📊 User: What's our total sales?")
print("\n🤖 Agent: Running query...")

result = bq_toolset.call("run_query", query="SELECT SUM(amount) as total_sales FROM sales")
print(f"\n💰 Total Sales: ${result['result'][0]['total_sales']:,}")


Simulating Data Analysis Agent

📊 User: What data do we have available?

🤖 Agent: Let me check the available datasets...
📡 McpToolset: Calling list_datasets on bigquery
✅ McpToolset: Got response from bigquery

📁 Available datasets: ['sales', 'customers']

📊 User: Tell me about the sales data

🤖 Agent: Getting schema for sales dataset...
📡 McpToolset: Calling get_schema on bigquery
✅ McpToolset: Got response from bigquery

📋 Sales dataset schema:
   Columns: ['date', 'product', 'amount']
   Rows: 3

📊 User: What's our total sales?

🤖 Agent: Running query...
📡 McpToolset: Calling run_query on bigquery
✅ McpToolset: Got response from bigquery

💰 Total Sales: $5,600


---
## 8. 完整的 ADK + MCP Agent 示例

以下是实际使用 ADK McpToolset 的完整代码示例：

In [12]:
# 完整的 ADK + MCP Agent 代码模板

complete_example = '''
# agent.py - Google Maps Assistant with MCP

import os
from google.adk.agents import LlmAgent
from google.adk.tools.mcp_tool import McpToolset
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

# 环境变量
GOOGLE_MAPS_API_KEY = os.getenv("GOOGLE_MAPS_API_KEY")

# System Prompt
MAPS_ASSISTANT_INSTRUCTION = """
You are a helpful maps and navigation assistant.

## Capabilities
- Get directions between locations
- Find places and businesses
- Provide distance and travel time estimates

## Guidelines
- Always confirm the origin and destination before getting directions
- Suggest alternative routes when available
- Mention traffic conditions if relevant
- Be helpful and concise
"""

# 创建 Agent with MCP Tools
root_agent = LlmAgent(
    model="gemini-2.0-flash",
    name="maps_assistant",
    description="A helpful assistant for maps and navigation",
    instruction=MAPS_ASSISTANT_INSTRUCTION,
    tools=[
        # MCP Toolset - 连接 Google Maps MCP 服务器
        McpToolset(
            connection_params=StdioConnectionParams(
                server_params=StdioServerParameters(
                    command="npx",
                    args=["-y", "@modelcontextprotocol/server-google-maps"],
                    env={"GOOGLE_MAPS_API_KEY": GOOGLE_MAPS_API_KEY}
                ),
            ),
            # 可选：过滤只使用特定工具
            tool_filter=["get_directions", "find_place", "search_nearby"]
        )
    ],
)

# 运行 Agent (使用 adk web 或 adk run)
# $ adk web agent.py
# $ adk run agent.py "How do I get from SF to LA?"
'''

print("Complete ADK + MCP Agent Example:")
print(complete_example)

Complete ADK + MCP Agent Example:

# agent.py - Google Maps Assistant with MCP

import os
from google.adk.agents import LlmAgent
from google.adk.tools.mcp_tool import McpToolset
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

# 环境变量
GOOGLE_MAPS_API_KEY = os.getenv("GOOGLE_MAPS_API_KEY")

# System Prompt
MAPS_ASSISTANT_INSTRUCTION = """
You are a helpful maps and navigation assistant.

## Capabilities
- Get directions between locations
- Find places and businesses
- Provide distance and travel time estimates

## Guidelines
- Always confirm the origin and destination before getting directions
- Suggest alternative routes when available
- Mention traffic conditions if relevant
- Be helpful and concise
"""

# 创建 Agent with MCP Tools
root_agent = LlmAgent(
    model="gemini-2.0-flash",
    name="maps_assistant",
    description="A helpful assistant for maps and navigation",
    instruction=MAPS_ASSISTANT_INSTRUCTION,
   

---
## 9. tool_filter：精确控制工具访问

In [13]:
# tool_filter 的重要性

print("为什么使用 tool_filter？")
print("=" * 50)
print("""
MCP 服务器可能暴露很多工具，但你的 Agent 可能只需要其中几个。

使用 tool_filter 的好处：

1. 🎯 减少上下文消耗
   - 每个工具的定义都占用 token
   - 工具越少，上下文越精简

2. 🔒 最小权限原则
   - 只暴露 Agent 需要的工具
   - 减少误用风险

3. 🚀 提高响应质量
   - LLM 选择工具时更准确
   - 减少"选择困难"
""")

# 示例对比
print("\n示例对比：")
print("-" * 50)

print("\n❌ 不使用 filter（暴露所有工具）:")
print("   Agent 看到: get_directions, find_place, search_nearby,")
print("               get_place_details, get_reviews, get_photos,")
print("               calculate_route, get_elevation, ...")
print("   问题: 太多选择，可能选错工具")

print("\n✅ 使用 filter（只暴露需要的）:")
print("   Agent 看到: get_directions, find_place")
print("   优点: 选择简单，不会误用")

为什么使用 tool_filter？

MCP 服务器可能暴露很多工具，但你的 Agent 可能只需要其中几个。

使用 tool_filter 的好处：

1. 🎯 减少上下文消耗
   - 每个工具的定义都占用 token
   - 工具越少，上下文越精简

2. 🔒 最小权限原则
   - 只暴露 Agent 需要的工具
   - 减少误用风险

3. 🚀 提高响应质量
   - LLM 选择工具时更准确
   - 减少"选择困难"


示例对比：
--------------------------------------------------

❌ 不使用 filter（暴露所有工具）:
   Agent 看到: get_directions, find_place, search_nearby,
               get_place_details, get_reviews, get_photos,
               calculate_route, get_elevation, ...
   问题: 太多选择，可能选错工具

✅ 使用 filter（只暴露需要的）:
   Agent 看到: get_directions, find_place
   优点: 选择简单，不会误用


---
## 10. Key Takeaways

### MCP 是什么

| 要点 | 说明 |
|------|------|
| **标准协议** | 由 Anthropic 开发，现已捐赠给 Linux Foundation |
| **USB-C for AI** | 让 Agent 可以连接任何兼容的工具 |
| **三种连接方式** | Stdio（本地）、SSE、StreamableHTTP（远程） |

### Google Managed MCP

| 特点 | 收益 |
|------|------|
| **托管服务** | 无需自己部署和运维 |
| **安全防护** | Cloud IAM + Model Armor |
| **企业级服务** | Maps, BigQuery, GKE 等 |

### ADK McpToolset

```python
# 基本用法
McpToolset(
    connection_params=...,  # Stdio/SSE/HTTP
    tool_filter=[...]       # 可选过滤
)

# 工作流程
1. 连接 MCP 服务器
2. 调用 list_tools 获取工具列表
3. 转换为 ADK 格式
4. 代理工具调用
5. 返回结果
```

### 最佳实践

| ✅ Do | ❌ Don't |
|-------|----------|
| 使用 tool_filter 限制工具 | 暴露所有工具给 Agent |
| 使用 Managed MCP（生产环境） | 自己部署服务器（如非必要） |
| 配置适当的 IAM 权限 | 使用过于宽松的权限 |
| 监控 MCP 调用和成本 | 忽略使用量和计费 |

---
## 11. Resources

### 官方文档
- [ADK MCP Documentation](https://google.github.io/adk-docs/mcp/) - ADK 中的 MCP 集成
- [ADK MCP Tools](https://google.github.io/adk-docs/tools-custom/mcp-tools/) - McpToolset 详细用法
- [Announcing Official MCP Support for Google Services](https://cloud.google.com/blog/products/ai-machine-learning/announcing-official-mcp-support-for-google-services) - 官方公告

### 示例代码
- [Launch My Bakery GitHub Example](https://github.com/google/mcp/tree/main/examples/launchmybakery) - 官方示例
- [Getting Started with MCP, ADK and A2A](https://codelabs.developers.google.com/codelabs/currency-agent) - Google Codelabs

### 深入阅读
- [Kaggle Whitepaper: Agent Tools and Interoperability with MCP](https://www.kaggle.com/whitepaper-agent-tools-and-interoperability-with-mcp)
- [Building ADK Agents using MCP Tools](https://medium.com/google-cloud/building-adk-v1-2-0-agents-using-mcp-tools-e97fb5e47961)
- [ADK meets MCP: Bridging Worlds of AI Agents](https://medium.com/google-cloud/adk-meets-mcp-bridging-worlds-of-ai-agents-1ed96ef5399c)
- [Use Google ADK and MCP with an external server](https://cloud.google.com/blog/topics/developers-practitioners/use-google-adk-and-mcp-with-an-external-server)

### 新闻报道
- [Google launches managed MCP servers](https://techcrunch.com/2025/12/10/google-is-going-all-in-on-mcp-servers-agent-ready-by-design/) - TechCrunch
- [Google Cloud goes all in on agentic AI protocol](https://www.sdxcentral.com/news/google-cloud-goes-all-in-on-agentic-ai-protocol-with-mcp-expansion/) - SDxCentral